In [1]:
pip install pandas tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy==1.24.4

  Obtaining dependency information for numpy==1.24.4 from https://files.pythonhosted.org/packages/d8/ec/ebef2f7d7c28503f958f0f8b992e7ce606fb74f9e891199329d5f5f87404/numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.8 MB 1.1 MB/s eta 0:00:14
    --------------------------------------- 0.3/14.8 MB 3.4 MB/s eta 0:00:05
   -- ------------------------------------- 1.1/14.8 MB 8.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.7/14.8 MB 16.9 MB/s eta 0:00:01
   ------- -------------------------------- 2.9/14.8 MB 15.7 MB/s eta 0:00:01
   -------------- ------------------------- 5.5/14.8 MB 23.5 MB/s eta 0:00:01
   ---------------- ----------------------- 6.3/14.8 MB 26.8 MB/s eta 0:00:01
   ---------------- ----------------------- 6.3/14.8 MB 26.8 MB/s eta 0:00:01
   ---------------- ------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.


In [3]:
pip install tensorflow --upgrade

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for h5py>=3.10.0 from https://files.pythonhosted.org/packages/b6/35/ed21094eb4d8acf31ccc7666a4d8701c1ce38f8d1fa3c7036f24416f6337/h5py-3.10.0-cp311-cp311-win_amd64.whl.metadata
  Using cached h5py-3.10.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Obtaining dependency information for ml-dtypes~=0.3.1 from https://files.pythonhosted.org/packages/a4/db/1784b87285588788170f87e987bfb4bda218d62a70a81ebb66c94e7f9b95/ml_dtypes-0.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorboar

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('feature_output_10k_final.csv')
df = df.sample(n=5000, random_state=42)

# Assuming all other columns except 'label' are features
# Define a mapping dictionary
mapping = {'Human': 0, 'AI': 1}

# Create a new column 'label_numeric' with mapped values
df['label_numeric'] = df['label'].map(mapping)
y = df['label_numeric']

non_numeric_columns = df.select_dtypes(exclude=['number']).columns
X = df.drop(non_numeric_columns, axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Manually normalize the feature data
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std  # Use the mean and std from training data for test data

# If you have NaNs in your dataset, you may want to handle them before normalization
# For example, replacing NaNs with zeros (assuming NaNs represent missing values)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

print(X)

print("Training set class distribution:")
print(y_train.value_counts())

print("\nTesting set class distribution:")
print(y_test.value_counts())

# Verify that the mean and standard deviation of the testing set features match those of the training set
print("Mean of X_test features after normalization:", X_test.mean(axis=0))
print("Standard deviation of X_test features after normalization:", X_test.std(axis=0))

      Unnamed: 0.1  Unnamed: 0  generated  length    neg    neu    pos  \
6252          6252       78992        1.0    4355  0.006  0.899  0.095   
4684          4684      357154        0.0    1156  0.010  0.867  0.122   
1731          1731      382833        1.0    2312  0.011  0.766  0.222   
4742          4742       64962        0.0    1285  0.102  0.759  0.139   
4521          4521      102689        1.0    1950  0.075  0.719  0.207   
...            ...         ...        ...     ...    ...    ...    ...   
5743          5743      188696        0.0    1754  0.014  0.848  0.138   
6226          6226       63866        0.0    1375  0.076  0.791  0.133   
1973          1973      394393        1.0    2129  0.004  0.774  0.221   
336            336      103411        0.0    2086  0.033  0.812  0.155   
2764          2764      429054        0.0    2401  0.071  0.860  0.069   

      compound  subjectivity_score  flesch_reading_ease_score  \
6252    0.9929            0.305413            

In [15]:
import tensorflow as tf
from tensorflow.keras import layers

#Define the model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

#Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

C:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8738 - loss: 0.3486 - val_accuracy: 1.0000 - val_loss: 0.0126
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9964 - loss: 0.0248 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9995 - loss: 0.0081 - val_accuracy: 1.0000 - val_loss: 3.9636e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 1.8280e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9996 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 7.1872e-05
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9987 - loss: 0.0040 - val_accuracy: 1.0000 - val_loss: 4.7935e-05
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 2.6013e-05
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9991 - loss: 0.003

In [17]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 1.0000 - loss: 6.9837e-06

Test accuracy: 1.0


In [19]:
#This neural network is only relying on the paragraphs of text to determine if Human or AI.

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load the data from the CSV file
data = pd.read_csv('feature_output_10k_final.csv')

# Split the data into features (X) and labels (y)
# Define a mapping dictionary
mapping = {'Human': 0, 'AI': 1}

# Create a new column 'label_numeric' with mapped values
df['label_numeric'] = df['label'].map(mapping)
y = df['label_numeric']

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X = df['text']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

# Pad sequences to ensure uniform length
max_length = 200  # Adjust this value based on the maximum length of your paragraphs
padded_sequences = pad_sequences(sequences, maxlen=max_length)

print("First 10 sequences:")
for sequence in sequences[:10]:
    print(sequence)

First 10 sequences:
[42, 6291, 5, 1, 233, 380, 579, 100, 3, 144, 1542, 6, 675, 229, 3, 5664, 426, 6, 1, 5345, 5, 195, 836, 610, 378, 1, 5173, 5, 970, 1235, 3, 233, 489, 8, 774, 10, 560, 1889, 545, 675, 2739, 3, 77, 5664, 892, 1, 65, 935, 1054, 2, 1687, 1, 9144, 5, 233, 489, 1, 233, 380, 579, 100, 584, 3737, 85, 28, 4, 2584, 3, 1214, 1405, 1248, 10, 1335, 1, 947, 5, 18, 424, 8, 2, 251, 42, 6291, 5, 584, 446, 144, 9145, 3, 5844, 141, 59, 5, 144, 1542, 6, 675, 770, 336, 3, 5664, 426, 427, 33, 13194, 2979, 2705, 4576, 3, 5346, 5845, 2169, 5665, 6, 7940, 1, 233, 380, 579, 100, 584, 8, 4, 8488, 4903, 217, 2, 6292, 3, 1687, 1, 4275, 793, 6, 233, 489, 584, 936, 17, 1, 1411, 7, 233, 1255, 15, 12, 2585, 16725, 141, 5035, 676, 1604, 1670, 1446, 767, 28, 380, 2468, 5846, 33, 378, 1, 5846, 1335, 15, 1296, 638, 489, 243, 1, 2740, 3, 3025, 4014, 1249, 3, 229, 52, 5, 1, 529, 205, 5, 208, 584, 8, 144, 551, 2, 1687, 233, 489, 778, 137, 1313, 28, 233, 489, 663, 4, 2221, 1235, 5, 229, 1, 584, 590, 8, 1207

In [26]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# Define the maximum length of the input sequences (i.e., the maximum number of words in each paragraph)
max_length = padded_sequences.shape[1]

# Define the size of the vocabulary (plus 1 to account for the padding token)
vocab_size = len(word_index) + 1

# Define the neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100))
model.add(LSTM(units=128))
model.add(Dense(units=1, activation='sigmoid'))

In [27]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - accuracy: 0.7160 - loss: 0.5481 - val_accuracy: 0.9470 - val_loss: 0.1673
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 77ms/step - accuracy: 0.9715 - loss: 0.0917 - val_accuracy: 0.8650 - val_loss: 0.3093
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.9816 - loss: 0.0770 - val_accuracy: 0.9620 - val_loss: 0.1559
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 77ms/step - accuracy: 0.9861 - loss: 0.0516 - val_accuracy: 0.9270 - val_loss: 0.2058
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.9932 - loss: 0.0250 - val_accuracy: 0.9460 - val_loss: 0.2346
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 75ms/step - accuracy: 0.9986 - loss: 0.0066 - val_accuracy: 0.9450 - val_loss: 0.2115
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 75ms/step - accuracy: 0.9983 - loss: 0.0096 - val_accuracy: 0.9470 - val_loss: 0.2202
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.9950 - loss: 0.0167 - val_a

In [28]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9405 - loss: 0.3229

Test accuracy: 0.9459999799728394
